In [151]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_validate
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing

import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

red_path = "./res/winequality-red.csv"
white_path = "./res/winequality-white.csv"
result_folder = "./results/"

test_set_ratio = 0.1

### Preprocessing

In [155]:
#df_red = pd.read_csv(red_path,";")
df_white = pd.read_csv(white_path, ";")

#df_ = pd.concat([df_red])
df_ = pd.concat([df_white])

### One of these

In [156]:
## add ID column
df_['quality'] = [0 if (x<=6) else x for x in df_['quality']]
df_['quality'] = [1 if (x>6) else x for x in df_['quality']]
df_ = df_.reset_index(drop=True)
df_['ID'] = df_.index
bip = df_.groupby(['quality']).agg(['count'])
bip

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,ID
,count,count,count,count,count,count,count,count,count,count,count,count
quality,,,,,,,,,,,,
0,3838,3838,3838,3838,3838,3838,3838,3838,3838,3838,3838,3838
1,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060,1060


In [146]:
df_['quality'] = [4 if (x==3 or x==4) else x for x in df_['quality']]
df_['quality'] = [8 if (x==8 or x==9) else x for x in df_['quality']]
df_['quality'] = df_['quality']-3
df_ = df_.reset_index(drop=True)
df_['ID'] = df_.index
bip = df_.groupby(['quality']).agg(['count'])
bip

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,ID
,count,count,count,count,count,count,count,count,count,count,count,count
quality,,,,,,,,,,,,
1,183,183,183,183,183,183,183,183,183,183,183,183
2,1457,1457,1457,1457,1457,1457,1457,1457,1457,1457,1457,1457
3,2198,2198,2198,2198,2198,2198,2198,2198,2198,2198,2198,2198
4,880,880,880,880,880,880,880,880,880,880,880,880
5,180,180,180,180,180,180,180,180,180,180,180,180


#### Center and Scale

In [157]:
cols = list(df_)
a = df_.values
b = preprocessing.scale(a[:,0:11], axis=0, with_mean=True, with_std=True, copy=True)

df_ = np.concatenate((b,df_.values[:,[11,12]]), axis=1)
df_ = pd.DataFrame(df_, columns = cols)
df_.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,ID
0,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152,0.0,0.0
1,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276,0.0,1.0
2,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667,0.0,2.0
3,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203,0.0,3.0
4,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203,0.0,4.0
5,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667,0.0,5.0
6,-0.776015,0.414339,-1.439511,0.120000,-0.035355,-0.312141,-0.055553,0.291789,-0.054751,-0.173921,-0.743008,0.0,6.0
7,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152,0.0,7.0
8,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276,0.0,8.0
9,1.475751,-0.577879,0.791757,-0.964483,-0.081131,-0.429751,-0.220283,-0.076030,0.210175,-0.349184,0.394746,0.0,9.0


#### Train/Test Split

In [158]:
## create train/test for prediction (not training it)
df_train, df_test = train_test_split(df_, shuffle=True, test_size=test_set_ratio, random_state=0)
df_train = df_train.sort_values(by=['ID']).reset_index(drop=True)
df_test = df_test.sort_values(by=['ID']).reset_index(drop=True)
df_train = df_train.drop('ID', axis=1)
df_test = df_test.drop('ID', axis=1)

In [159]:
df_train.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152,0.0
1,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276,0.0
2,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667,0.0
3,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203,0.0
4,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203,0.0
5,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667,0.0
6,-0.776015,0.414339,-1.439511,0.120000,-0.035355,-0.312141,-0.055553,0.291789,-0.054751,-0.173921,-0.743008,0.0
7,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152,0.0
8,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276,0.0
9,1.475751,-0.577879,0.791757,-0.964483,-0.081131,-0.429751,-0.220283,-0.076030,0.210175,-0.349184,0.394746,0.0


### Initialize Training data

In [106]:
df_vars = df_train.loc[:, df_train.columns != 'quality']
df_class = df_train['quality']

X_train, X_test, y_train, y_test = train_test_split(df_vars, df_class, test_size=test_set_ratio, random_state=42)
#clf = SVC(gamma='auto')
clf = LinearSVC()
#clf = GaussianNB()

### Simple cross validation for well-founded results

In [107]:
scores = cross_validate(clf, df_vars, df_class, cv=10)
scores['test_score']

array([0.46575342, 0.60958904, 0.54109589, 0.42465753, 0.53424658,
       0.64583333, 0.62676056, 0.59574468, 0.61702128, 0.56028369])

In [16]:
min_samples_splits = [2,3,4,5,7,10,15,20,25]
results = []
for min_samples_split in min_samples_splits:
    #clf = DecisionTreeClassifier(min_samples_split=min_samples_split)
    clf = SVC(gamma='auto')
    scores = cross_validate(clf, df_vars, df_class, cv=5)
    results.append(scores['test_score'].mean())
    
plt.plot(min_samples_splits, results, "r", label="Test Accuracy")
plt.ylabel("accuracy")
plt.xlabel("min_samples_split")
plt.title("Mean 5-fold cross-validated accuracy")
plt.show()

NameError: name 'SVC' is not defined

### Model fitting and sample testing

In [31]:
model = clf.fit(X_train, y_train)
model.score(X_test, y_test)

0.5763888888888888

### Final prediction

In [32]:
df_res = pd.DataFrame()
df_res['ID'] = df_test['ID']
df_res['type'] = pd.Series(model.predict(df_test.loc[:,df_test.columns != 'type']))
df_res.head(10)
#np.count_nonzero(df_res['quality']==df_test['quality'])

ValueError: X has 13 features per sample; expecting 12

### Export

In [ ]:
export_file_name = result_folder + "simple_svm.csv"
df_res.to_csv(export_file_name, index=False)

### Easy comparison

In [160]:
import traceback
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors.nearest_centroid import NearestCentroid

df_vars = df_train.loc[:, df_train.columns != 'quality']
df_class = df_train['quality']

# Adjust size here - Terminates in reasonable time for 10.000 elements and k<=40 attributes
# Won't terminate for 100.000
#df_vars=df_vars.head(5000)
#df_class=df_class.head(5000)
#df_class[df_class<0] = 0
#df_vars = np.where(df_vars < 0, 2, df_vars)

#X, y = df_vars,df_class
##Adjust k attributes here
#X_new = SelectKBest(chi2, k=1).fit_transform(X, y)
#df_vars=X_new
#X_train, X_test, y_train, y_test = train_test_split(X_new, df_class, test_size=test_set_ratio, random_state=0)

names = ["Nearest Centroid", "Linear SVM",
         "Decision Tree", "Naive Bayes"]

classifiers = [
    NearestCentroid(),
    LinearSVC(),
    DecisionTreeClassifier(),
    GaussianNB()]

cl_scores = {}

for name, clf in zip(names, classifiers):
    try:
        np.random.seed(32143421)
        scores = cross_validate(clf, df_vars, df_class, cv=10, scoring='accuracy', return_train_score=True)
        cl_scores[name] = scores
        print(name,"-- Training Set --", "Mean", scores['train_score'].mean(), "-- Min", scores['train_score'].min(), "-- Max", scores['train_score'].max())
        print(name,"-- Test Set --", "Mean", scores['test_score'].mean(), "-- Min", scores['test_score'].min(), "-- Max", scores['test_score'].max())
    except:
        print("Classification failed for", name)
        traceback.print_exc()

Nearest Centroid -- Training Set -- Mean 0.6745307220231755 -- Min 0.6694402420574886 -- Max 0.680695564516129
Nearest Centroid -- Test Set -- Mean 0.670045328154572 -- Min 0.5659090909090909 -- Max 0.7782805429864253
Linear SVM -- Training Set -- Mean 0.810218934781733 -- Min 0.8054435483870968 -- Max 0.8127520161290323
Linear SVM -- Test Set -- Mean 0.8067121347373447 -- Min 0.7727272727272727 -- Max 0.8363636363636363
Decision Tree -- Training Set -- Mean 1.0 -- Min 1.0 -- Max 1.0
Decision Tree -- Test Set -- Mean 0.744749568825199 -- Min 0.6848072562358276 -- Max 0.7959183673469388
Naive Bayes -- Training Set -- Mean 0.7268094014634714 -- Min 0.7217040584824804 -- Max 0.7328629032258065
Naive Bayes -- Test Set -- Mean 0.7260890300176015 -- Min 0.6386363636363637 -- Max 0.8095238095238095


In [ ]:
traces = []
for clf_name in names:
    traces.append(go.Box(
        y=cl_scores[clf_name]['test_score'],
        boxpoints='all',
        name=clf_name
    ))
    
layout = go.Layout(title="Accuracy on Test Set (Wine Quality)")
    
py.iplot(go.Figure(data=traces, layout=layout))